In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import dqn
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

#from models import VisualEncoder
from train import *
from wrappers_2 import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4, padding=0),  
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(), 
            nn.Conv2d(64, 512, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [3]:
class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 512
        self.encoder = VisualEncoder()
        self.encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AnnaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.qvalue_head = nn.Linear(features_dim, num_outputs)
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.qvalue_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
            
        features = self.encoder(obs)
        qvalues = self.qvalue_head(features)
        return qvalues, state

In [4]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def reward(self, rew):
        if rew == 0:
            rew = -0.05
        return rew
    
def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=500)
    env.update_taskset(TaskSet(preset=['C8']))
    env = PovOnlyWrapper(env)
    env = SelectAndPlace(env)
    env = Discretization(env, flat_action_space('human-level'))
    #env = RewardWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.dqn import ApexTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

analysis = tune.run(ApexTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "gamma": 0.95,
             "num_gpus": 1,
             "num_workers": 3,
             "buffer_size": 200000,
             "learning_starts": 5000,
             "train_batch_size": 5000,
             "target_network_update_freq": 10000,
             "prioritized_replay_alpha": 0.5,
             "final_prioritized_replay_beta": 1.0,
             "min_iter_time_s": 10,
             "rollout_fragment_length": 8,
             "collect_metrics_timeout": 1800,
             
             "v_min": -20.0,
             "v_max": 20.0,
             
             "exploration_config": {
                  "initial_epsilon": 1,
                  "epsilon_timesteps": 500000,
                  "final_epsilon": 0.05,
              },
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "APEX C8 pretrained (AnnaCNN) gamma: 0.95"
                  }
              }

        },
        loggers=[WandbLogger])

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)
2021-10-13 08:14:27,555	INFO wandb.py:170 -- Already logged into W&B.
2021-10-13 08:14:27,568	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


Trial name,status,loc
APEX_my_env_95522_00000,RUNNING,


wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
wandb: WARNING Tried to auto resume run with id 9c652_00000 but id 95522_00000 is set.
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
(pid=152) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=152)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=152) 2021-10-13 08:14:31,038	INFO dqn.py:188 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=152) 2021-10-13 08:14:31,038	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=147) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=147)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=149) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=149)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=150) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=150)   w

Result for APEX_my_env_95522_00000:
  agent_timesteps_total: 25040
  custom_metrics: {}
  date: 2021-10-13_08-21-10
  done: false
  episode_len_mean: 218.6216216216216
  episode_media: {}
  episode_reward_max: 3.0
  episode_reward_mean: 0.6756756756756757
  episode_reward_min: -1.0
  episodes_this_iter: 37
  episodes_total: 37
  experiment_id: 761a764a1328456a9969c858fe012428
  hostname: linar-Z390-GAMING-X
  info:
    exploration_infos:
    - cur_epsilon: 0.0
      last_timestep: 0
    - cur_epsilon: 0.4
      last_timestep: 24215
    - cur_epsilon: 0.016190861620062107
      last_timestep: 24863
    - cur_epsilon: 0.0006553600000000003
      last_timestep: 24551
    last_target_update_ts: 6525000
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_lr: 0.0005
        grad_gnorm: 0.2605246603488922
        max_q: 10.484436988830566
        mean_q: 1.2949919700622559
        min_q: -0.9507524967193604
    learner_queue:
      size_count: 1307
      size_mean: 0

...

Result for APEX_my_env_95522_00000:
  agent_timesteps_total: 375208
  custom_metrics: {}
  date: 2021-10-13_09-42-57
  done: false
  episode_len_mean: 492.95
  episode_media: {}
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 21
  episodes_total: 539
  experiment_id: 761a764a1328456a9969c858fe012428
  hostname: linar-Z390-GAMING-X
  info:
    exploration_infos:
    - cur_epsilon: 0.0
      last_timestep: 0
    - cur_epsilon: 0.4
      last_timestep: 374735
    - cur_epsilon: 0.016190861620062107
      last_timestep: 375167
    - cur_epsilon: 0.0006553600000000003
      last_timestep: 374791
    last_target_update_ts: 103185000
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_lr: 0.0005
        grad_gnorm: 76385.640625
        max_q: 69443712.0
        mean_q: 4168082.0
        min_q: -2478.599853515625
    learner_queue:
      size_count: 20638
      size_mean: 0.6
      size_quantiles:
      - 0.0
      

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
APEX_my_env_95522_00000,RUNNING,192.168.3.5:152,15,5298.79,375208,0,0,0,492.95


(pid=149) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=149)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=147) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=147)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Result for APEX_my_env_95522_00000:
  agent_timesteps_total: 400208
  custom_metrics: {}
  date: 2021-10-13_09-46-58
  done: false
  episode_len_mean: 490.38
  episode_media: {}
  episode_reward_max: 0.0
  episode_reward_mean: 0.0
  episode_reward_min: 0.0
  episodes_this_iter: 18
  episodes_total: 557
  experiment_id: 761a764a1328456a9969c858fe012428
  hostname: linar-Z390-GAMING-X
  info:
    exploration_infos:
    - cur_epsilon: 0.0
      last_timestep: 0
    - cur_epsilon: 0.4
      last_timestep: 399607
    - cur_epsilon: 0.016190861620062107
      last_timestep: 399831
    - cur_epsilon: 0.0006553600000000003
      last_timestep: 399863
    last_target_update_ts: 107295000
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_lr: 0.0005
        grad_gnorm: 5459.8154296875
        max_q: 32462.51171875
        mean_q: 406.15838623046875
        min_q: -1212.914794921875
    learner_queue:
      size_count: 21462
      size_mean: 0.44
      size_quantiles:
 

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
APEX_my_env_95522_00000,RUNNING,192.168.3.5:152,16,5540.53,400208,0,0,0,490.38
